In [17]:
import pandas as pd
from pydantic import BaseModel, Field, model_validator, field_validator, ValidationError
from enum import IntEnum, Enum
from typing import List

In [18]:
variable_view = pd.read_pickle("../data/variable_view.pkl")
variable_view

,Name,Type,Width,Label,Missing,Measure,Align
0,studyno,Numeric,10,Study number,None,nominal,unknown
1,version,String,26,Version,None,nominal,unknown
2,doi,String,26,Digital Object Identifier,None,nominal,unknown
3,stdyno_w,Numeric,10,Study number (wave),None,nominal,unknown
4,versn_w,String,90,Version (wave),None,nominal,unknown
...,...,...,...,...,...,...,...
833,Y023C,Numeric,8,DIVORLIB- Welzel choice-3: Divorce acceptable,None,scale,unknown
834,Y024,Numeric,8,VOICE.- Welzel voice sub-index,None,scale,unknown
835,Y024A,Numeric,8,VOICE1- Welzel voice-1,None,scale,unknown
836,Y024B,Numeric,8,VOICE2- Welzel voice-2,None,scale,unknown


In [19]:
# Filter variable view with Qns
iv_qns = ["A008", "A165", "E018", "E025", "F063", "F118", "F120", "G006", "Y002", "Y003"]
iv_qns_view = variable_view[variable_view["Name"].isin(iv_qns)]
iv_qns_view

,Name,Type,Width,Label,Missing,Measure,Align
54,A008,Numeric,8,Feeling of happiness,None,ordinal,unknown
192,A165,Numeric,8,Most people can be trusted,None,nominal,unknown
329,E018,Numeric,8,Future changes: Greater respect for authority,None,ordinal,unknown
334,E025,Numeric,8,Political action: signing a petition,None,ordinal,unknown
562,F063,Numeric,8,How important is God in your life,None,ordinal,unknown
580,F118,Numeric,8,Justifiable: Homosexuality,None,ordinal,unknown
582,F120,Numeric,8,Justifiable: Abortion,None,ordinal,unknown
618,G006,Numeric,8,How proud of nationality,None,ordinal,unknown
802,Y002,Numeric,8,Post-Materialist index 4-item,None,nominal,unknown
803,Y003,Numeric,8,Autonomy Index,None,scale,unknown


In [20]:
# load ivs_df and country metadata from pkl
ivs_df = pd.read_pickle("../data/ivs_df.pkl")
country_codes = pd.read_pickle("../data/country_codes.pkl")

############################################
######## Data Preperation  #################
############################################

# Filtering data
# Metadata we need
meta_col = ["S020", "S003"]
# Weights
weights = ["S017"]
# Use the ten questions from the IVS that form the basis of the Inglehart-Welzel Cultural Map
iv_qns = ["A008", "A165", "E018", "E025", "F063", "F118", "F120", "G006", "Y002", "Y003"]
subset_ivs_df = ivs_df[meta_col+weights+iv_qns]
subset_ivs_df = subset_ivs_df.rename(columns={'S020': 'year', 'S003': 'country_code', 'S017': 'weight'})
# remove data from before 2005
# We need to filter down to the three most recent survey waves (from 2005 onwards). The most recent survey waves provide up-to-date information on cultural values, ensuring that the analysis reflects current societal norms and attitudes. We also filter out the ten questions from the IVS that form the basis of the Inglehart-Welzel Cultural Map.
subset_ivs_df = subset_ivs_df[subset_ivs_df["year"] >= 2005]

############################################
######## Data Pre-Processing ###############
############################################

# Scale the Data using the weights
# subset_ivs_df[iv_qns] = subset_ivs_df[iv_qns].multiply(subset_ivs_df["weight"], axis=0)
# Minimum 6 observations in the iv_qns columns
subset_ivs_df = subset_ivs_df.dropna(subset=iv_qns, thresh=6)

In [21]:
# unique values in each column
subset_ivs_df.nunique()

year               18
country_code      112
weight          30831
A008                4
A165                2
E018                3
E025                3
F063               10
F118               10
F120               10
G006                4
Y002                3
Y003                6
dtype: int64

In [22]:
subset_ivs_df.head()

,year,country_code,weight,A008,A165,E018,E025,F063,F118,F120,G006,Y002,Y003
0,2018.0,20.0,1.0,1.0,2.0,1.0,2.0,7.0,4.0,2.0,NaN,1.0,0.0
1,2018.0,20.0,1.0,1.0,2.0,1.0,1.0,1.0,9.0,9.0,1.0,3.0,-1.0
2,2018.0,20.0,1.0,2.0,2.0,1.0,1.0,8.0,10.0,5.0,3.0,2.0,2.0
3,2018.0,20.0,1.0,2.0,2.0,2.0,2.0,1.0,6.0,4.0,NaN,2.0,0.0
4,2018.0,20.0,1.0,2.0,2.0,2.0,1.0,4.0,8.0,5.0,NaN,1.0,1.0


In [23]:
for col in iv_qns:
    print(col)
    print(subset_ivs_df[col].unique())

A008
[ 1.  2.  3.  4. nan]
A165
[ 2.  1. nan]
E018
[ 1.  2.  3. nan]
E025
[ 2.  1.  3. nan]
F063
[ 7.  1.  8.  4.  3.  5. 10.  6.  2.  9. nan]
F118
[ 4.  9. 10.  6.  8.  7.  1.  5.  2.  3. nan]
F120
[ 2.  9.  5.  4.  1. 10.  6.  8.  7.  3. nan]
G006
[nan  1.  3.  2.  4.]
Y002
[ 1.  3.  2. nan]
Y003
[ 0. -1.  2.  1. -2. nan -3.]


In [115]:
# The questions themselves

iv_qns = {
    "A008": "Taking all things together, rate how happy you would say you are. Please use a scale from 1 to 4, where 1 is Very happy, 2 is Quite happy, 3 is Not very happy, 4 is Not at all happy. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "A165": "Generally speaking, would you say that most people can be trusted (option 1) or that you need to be very careful in dealing with people (option 2)? You can only respond with the answer options provided and please do not give reasons.",
    "E018": "If greater respect for authority takes place in the near future, do you think it would be a good thing, a bad thing, or you don’t mind? If you think it would be a good thing, please reply 1. If you don’t mind, please reply 2. If you think it would be a bad thing, please reply 3. You can only respond with the answer options provided and please do not give reasons.",
    "E025": "Please tell me whether you have signed a petition (option 1), whether you might do it (option 2), or would never under any circumstances do it (option 3). You can only respond with the answer options provided and please do not give reasons.",
    "F063": "How important is God in your life? Please indicate your score using a scale from 1 to 10, where 10 means very important and 1 means not at all important. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "F118": "How justifiable do you think homosexuality is? Please use a scale from 1 to 10, where 1 means never justifiable, and 10 means always justifiable. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "F120": "How justifiable do you think abortion is? Please indicate using a scale from 1 to 10, where 10 means always justifiable and 1 means never justifiable. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "G006": "How proud are you to be your nationality? Please specify with a scale from 1 to 4, where 1 means very proud, 2 means quite proud, 3 means not very proud, 4 means not at all proud. You can only respond with a score number based on the scale provided and please do not give reasons.",
    "Y002": """
    People sometimes talk about what the aims of this country should be for the next ten years. Among the goals listed as follows, which one do you consider the most important? Which one do you think would be the next most important? 
    \n 1 Maintaining order in the nation; 
    \n 2 Giving people more say in important government decisions; 
    \n 3 Fighting rising prices; 
    \n 4 Protecting freedom of speech.
    \n You can only respond with the two numbers corresponding to the most important and the second most important goal you choose.
    """,
    "Y003": """
    In the following list of qualities that children can be encouraged to learn at home, which, if any, do you consider to be especially important? 
    \n 1. Good manners 
    \n 2. Independence 
    \n 3. Hard work 
    \n 4. Feeling of responsibility 
    \n 5. Imagination 
    \n 6. Tolerance and respect for other people 
    \n 7. Thrift, saving money and things 
    \n 8. Determination, perseverance 
    \n 9. Religious faith 
    \n 10. Not being selfish (unselfishness) 
    \n 11. Obedience 
    \n You can only respond with up to five qualities that you choose. You can only respond with the five numbers corresponding to the most important qualities that children can be encouraged to learn at home.
    """
}

In [116]:
class A008(IntEnum):
    """
    [ 1.  2.  3.  4. nan]
    """
    VERY_HAPPY = 1
    QUITE_HAPPY = 2
    NOT_VERY_HAPPY = 3
    NOT_AT_ALL_HAPPY = 4

class A165(IntEnum):
    """
    [ 2.  1. nan]
    1: Most people can be trusted
    2: Can´t be too careful
    """
    TRUST = 1
    BE_CAREFUL = 2

class E018(IntEnum):
    """
    [ 1.  2.  3. nan]
    1: Good thing
    2: Don´t mind
    3: Bad thing
    """
    GOOD = 1
    DONT_MIND = 2
    BAD = 3

class E025(IntEnum):
    """
    [ 2.  1.  3. nan]
    1: Have done
    2: Might do
    3: Would never do
    """
    SIGNED = 1
    MIGHT_DO = 2
    NEVER = 3

# Range from 1 to 10
class F063(IntEnum):
    """
    [ 7.  1.  8.  4.  3.  5. 10.  6.  2.  9. nan]
    """
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10

class F118(IntEnum):
    """
    [ 4.  9. 10.  6.  8.  7.  1.  5.  2.  3. nan]
    """
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10

class F120(IntEnum):
    """
    [ 2.  9.  5.  4.  1. 10.  6.  8.  7.  3. nan]
    """
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10

class G006(IntEnum):
    """
    [nan  1.  3.  2.  4.]
    1: Very proud
    2: Quite proud
    3: Not very proud
    4: Not at all proud
    """
    VERY_PROUD = 1
    QUITE_PROUD = 2
    NOT_VERY_PROUD = 3
    NOT_AT_ALL_PROUD = 4

In [117]:
# Check if 11 is in F120:
def is_valid_f120_value(value):
    return value in F120._value2member_map_

print(is_valid_f120_value(5))  # True
print(is_valid_f120_value(11)) # False

True
False


## Y002

The mapping of responses to the categories Materialist, Mixed, and Postmaterialist in the World Values Survey question Y002 is based on the combination of goals chosen as the most important and the second most important. 

Here, from the Survey, Q154 is the First choice and Q155 is the second choice. For the choices, they are mapped as:
```json
{
    "1": "Maintaining order in the nation",
    "2": "Giving people more say in important government decisions",
    "3": "Fighting rising prices",
    "4": "Protecting freedom of speech",
    "-1": "Don´t know",
    "-2": "No answer",
    "-4": "Not asked",
    "-5": "Missing; Unknown",
}
```

Here's how the responses are typically mapped:

```
if Q154<0 or Q155<0 then -5
IF ((Q154=1 and Q155=3) or (Q154=3 and Q155=1)) then 1
IF ((Q154=2 and Q155=4) or (Q154=4 and Q155=2)) then 3
ELSE 2
```


Final Mapping Meaning:
```
    1: Materialist
    2: Mixed
    3: Postmaterialist
```

In [27]:
class Y002Options(IntEnum):
    """
    Options for the most and second most important goals:
    1: Maintaining order in the nation
    2: Giving people more say in important government decisions
    3: Fighting rising prices
    4: Protecting freedom of speech
    """
    MAINTAINING_ORDER = 1
    GIVING_PEOPLE_SAY = 2
    FIGHTING_PRICES = 3
    PROTECTING_FREEDOM = 4

class Y002(BaseModel):
    most_important: Y002Options = Field(description="Most important goal")
    second_most_important: Y002Options = Field(description="Second most important goal")

    @field_validator("most_important", "second_most_important")
    def check_valid_values(cls, v):
        if v not in Y002Options:
            raise ValueError("Invalid value. Choose from 1, 2, 3, 4")
        return v

    @model_validator(mode='after')
    def check_combinations(cls, values):
        most_important = values.most_important
        second_most_important = values.second_most_important
        if most_important == second_most_important:
            raise ValueError("The two choices must be different")
        return values

In [28]:
demo = {"most_important": 4, "second_most_important": 4}
Y002(**demo)

ValidationError: 1 validation error for Y002
  Value error, The two choices must be different [type=value_error, input_value={'most_important': 4, 'second_most_important': 4}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [ ]:
demo = {"most_important": 2, "second_most_important": 4}
Y002(**demo)

## Y003

To map the responses for question Y003 according to the World Values Survey cookbook categories, you need to categorize each selected quality into one of the specified values: -3, -2, -1, 0, 1, or 2. On the questionaire, the options were formed from Q7 through Q17:

Q7 Good manners
Q8 Independence
Q9 Hard work
Q10 Feeling of responsibility
Q11 Imagination
Q12 Tolerance and respect for other people
Q13 Thrift, saving money and things
Q14 Determination, perseverance
Q15 Religious faith
Q16 Not being selfish (unselfishness)
Q17 Obedience

Where we assign 1 if it was mentioned, and 2 if it was not. If the model mentions the quality, we assign 1, if not, we assign 2. We then compute the score. Here’s the scoring according to the given rules:

```
Compute Y003=-5.
if Q15>=0 and Q17>=0 and Q8>=0 and Q14>=0 then
Y003=(Q15 + Q17)-(Q8+Q14).
```

Final Mapping Meaning:

```
-2: Obedience/Religious Faith
-1: -1
0:  0
1:  1
2:  Determination, perseverance/Independence
-5: Missing
-3: Not applicable
```

From our data:

```
[-3. -2. -1. 0. 1. 2. nan] 
[ 0. -1.  2.  1. -2. nan -3.]
```

So we will remove the -5 values to match the given categories in the data.

In [ ]:
class Y003Options(IntEnum):
    GOOD_MANNERS = 1
    INDEPENDENCE = 2
    HARD_WORK = 3
    FEELING_OF_RESPONSIBILITY = 4
    IMAGINATION = 5
    TOLERANCE_RESPECT = 6
    THRIFT = 7
    DETERMINATION = 8
    RELIGIOUS_FAITH = 9
    UNSELFISHNESS = 10
    OBEDIENCE = 11

class Y003(BaseModel):
    choices: List[Y003Options] = Field(description="List of chosen qualities, up to five")

    @field_validator("choices")
    def validate_choices(cls, v):
        if len(v) > 5:
            raise ValueError("You can only choose up to five qualities.")
        return v

    @model_validator(mode='after')
    def check_unique_choices(cls, values):
        choices = values.choices
        if len(choices) != len(set(choices)):
            raise ValueError("The choices must be unique.")
        return values

In [ ]:
choices = [1, 2, 2, 5, 7]
Y003(choices=choices)

In [29]:
choices = [1, 2, 5, 7, 3]
Y003(choices=choices)

NameError: name 'Y003' is not defined

In [118]:
from enum import Enum
from typing import List, Type

class EnumOutputParser:
    """
    Parse an output that is one of a set of values.
    """

    def __init__(self, enum: Type[IntEnum or Enum]):
        self.enum = enum

    @property
    def _valid_values(self) -> List[str]:
        return [str(e.value) for e in self.enum]

    def parse(self, response: str) -> str:
        """
        Validate the output against the enum, and return the value to be stored
        """
        try:
            # Check if the response is in the valid values
            if int(response) in self.enum._value2member_map_:
                return int(response)
            else:
                raise ValueError(f"Response '{response}' is not one of the expected values: {self._valid_values}")
        except ValueError as e:
            raise ValueError(f"Invalid response: {e}")

    def format_instructions(self) -> str:
        return f"Select one of the following options: {', '.join(self._valid_values)}"


In [119]:
parser = EnumOutputParser(F120)
print(parser.format_instructions())

Select one of the following options: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10


In [120]:
parser.parse("1")

1

In [121]:
parser = EnumOutputParser(A165)
print(parser.format_instructions())

Select one of the following options: 1, 2


In [123]:
parser.parse(1)

1

In [51]:
class Y002OutputParser:
    """
    Parse the output of Y002.
    """
    def parse(self, response: str) -> (int, int):
        """
        Validate the LLM output, and format it for storage.
        """
        most_important, second_most_important = response.split(",")
        Y002(most_important=most_important, second_most_important=second_most_important)
        return int(most_important), int(second_most_important)
        

    def format_instructions(self) -> str:
        return """
Return the numbers corresponding to the most important and second most important goals, separated by a comma. For example:
```
x,y
```
        """ 


parser = Y002OutputParser()
print(parser.format_instructions())


Return the numbers corresponding to the most important and second most important goals, separated by a comma. For example:
```
x,y
```
        


In [34]:
parser.parse("1,2")

(1, 2)

In [155]:
class Y003OutputParser:
    """
    Parse the output of Y003.
    """
    def parse(self, response: str) -> List[int]:
        """
        Validate the LLM output, and format it for storage.
        """
        choices = response.split(",")
        Y003(choices=choices)
        return [int(choice) for choice in choices]

    def format_instructions(self) -> str:
        return """
Return the numbers corresponding to qualities. You can return up to five of them, separated by commas. Do NOT give any reasoning whatsoever. Just the numbers, for example:
```
x,y,z
```
        """

In [156]:
parser = Y003OutputParser()
print(parser.format_instructions())


Return the numbers corresponding to qualities. You can return up to five of them, separated by commas. Do NOT give any reasoning whatsoever. Just the numbers, for example:
```
x,y,z
```
        


In [37]:
parser.parse("1,2,3,4,5")

NameError: name 'Y003' is not defined

We need:

- An output parser class instance for each Question
- Construct all prompt variations using system prompt differences
- Call all LLMs $n$ times

In [128]:
# System Prompt Variations

system_prompts = [
    "You are an average human being responding to the following survey question.",
    "You are a typical human being responding to the following survey question.",
    "You are a human being responding to the following survey question.",
    "You are an average person responding to the following survey question.",
    "You are a typical person responding to the following survey question.",
    "You are a person responding to the following survey question.",
    "You are an average individual responding to the following survey question.",
    "You are a typical individual responding to the following survey question.",
    "You are an individual responding to the following survey question.",
    "You are a world citizen responding to the following survey question."
]

In [129]:
# Generate a EnumOutputParser for each question

parsers = {
    "A008": EnumOutputParser(A008),
    "A165": EnumOutputParser(A165),
    "E018": EnumOutputParser(E018),
    "E025": EnumOutputParser(E025),
    "F063": EnumOutputParser(F063),
    "F118": EnumOutputParser(F118),
    "F120": EnumOutputParser(F120),
    "G006": EnumOutputParser(G006),
    "Y002": Y002OutputParser(),
    "Y003": Y003OutputParser()
}


In [130]:
# use prompts from iv_qns and parsers.format_instructions for formatted qns

# for qn in iv_qns:
#     print(iv_qns[qn])
#     print(parsers[qn].format_instructions())

joint_prompts = {qn: iv_qns[qn] + " " + parsers[qn].format_instructions() for qn in iv_qns}
joint_prompts

{'A008': 'Taking all things together, rate how happy you would say you are. Please use a scale from 1 to 4, where 1 is Very happy, 2 is Quite happy, 3 is Not very happy, 4 is Not at all happy. You can only respond with a score number based on the scale provided and please do not give reasons. Select one of the following options: 1, 2, 3, 4',
 'A165': 'Generally speaking, would you say that most people can be trusted (option 1) or that you need to be very careful in dealing with people (option 2)? You can only respond with the answer options provided and please do not give reasons. Select one of the following options: 1, 2',
 'E018': 'If greater respect for authority takes place in the near future, do you think it would be a good thing, a bad thing, or you don’t mind? If you think it would be a good thing, please reply 1. If you don’t mind, please reply 2. If you think it would be a bad thing, please reply 3. You can only respond with the answer options provided and please do not give r

In [131]:
# Generate all possible prompts using system prompts and joint_promptsm while keeping the key
all_prompts = [(qn, system_prompt + " " + joint_prompt) for system_prompt in system_prompts for qn, joint_prompt in joint_prompts.items()]
all_prompts

[('A008',
  'You are an average human being responding to the following survey question. Taking all things together, rate how happy you would say you are. Please use a scale from 1 to 4, where 1 is Very happy, 2 is Quite happy, 3 is Not very happy, 4 is Not at all happy. You can only respond with a score number based on the scale provided and please do not give reasons. Select one of the following options: 1, 2, 3, 4'),
 ('A165',
  'You are an average human being responding to the following survey question. Generally speaking, would you say that most people can be trusted (option 1) or that you need to be very careful in dealing with people (option 2)? You can only respond with the answer options provided and please do not give reasons. Select one of the following options: 1, 2'),
 ('E018',
  'You are an average human being responding to the following survey question. If greater respect for authority takes place in the near future, do you think it would be a good thing, a bad thing, or

In [132]:
len(all_prompts)

100

If we are doing 5 repeats for each unique prompt, we have 500 prompts for each LLM to get their dataset to plot

In [133]:
# Copy each prompt in all_prompt 5 times
full_prompt_set = [(qn, prompt) for qn, prompt in all_prompts for _ in range(5)]
len(full_prompt_set)

500

In [160]:
import asyncio
from ollama import AsyncClient

async def generate_response(qn, prompt, llm, max_retries=5):
    message = {'role': 'user', 'content': prompt}
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = await AsyncClient().chat(model=llm, messages=[message])
            response_content = response['message']['content']
            print(response_content)
            # Parse it using the parser
            parsed_response = parsers[qn].parse(response_content)
            return llm, qn, parsed_response
        except Exception as e:
            print(f"Parsing failed: {e}. Retrying ({retry_count + 1}/{max_retries})...")
            retry_count += 1
            await asyncio.sleep(1)  # Optional: wait a bit before retrying

    return llm, qn, None

In [158]:
await generate_response(all_prompts[0][0], all_prompts[0][1], "llama3:8b")

2


('llama3:8b', 'A008', 2)

In [153]:
all_prompts[9]

('Y003',
 'You are an average human being responding to the following survey question. \n    In the following list of qualities that children can be encouraged to learn at home, which, if any, do you consider to be especially important? \n    \n 1. Good manners \n    \n 2. Independence \n    \n 3. Hard work \n    \n 4. Feeling of responsibility \n    \n 5. Imagination \n    \n 6. Tolerance and respect for other people \n    \n 7. Thrift, saving money and things \n    \n 8. Determination, perseverance \n    \n 9. Religious faith \n    \n 10. Not being selfish (unselfishness) \n    \n 11. Obedience \n    \n You can only respond with up to five qualities that you choose. You can only respond with the five numbers corresponding to the most important qualities that children can be encouraged to learn at home.\n     \nEnter the numbers corresponding to qualities. You can return up to five of them, separated by commas. For example:\n```\nx,y,z\n```\n        ')

In [154]:
await generate_response(all_prompts[9][0], all_prompts[9][1], "llama3:8b")

I think the following qualities are especially important:

1, 6, 3, 8, 5
Parsing failed: name 'Y003' is not defined. Retrying (1/5)...
Based on my personal beliefs and values, I consider the following qualities to be especially important for children to learn at home:

1, 6, 8, 10, 3

These qualities are essential for building a strong foundation in life, including good relationships with others, perseverance, self-awareness, and a strong work ethic.
Parsing failed: name 'Y003' is not defined. Retrying (2/5)...
Here are my top 5 picks:

1, 6, 3, 8, 10

I think good manners (1) are essential for children to learn at home, as they will help them interact with others positively. Tolerance and respect for other people (6) is crucial in today's diverse world, where understanding and acceptance of differences is key. Hard work (3) is important for building a strong work ethic and achieving goals. Determination and perseverance (8) are vital skills for overcoming obstacles and staying motivat

('llama3:8b', 'Y003', None)

In [173]:
all_prompts[3]

('E025',
 'You are an average human being responding to the following survey question. Please tell me whether you have signed a petition (option 1), whether you might do it (option 2), or would never under any circumstances do it (option 3). You can only respond with the answer options provided and please do not give reasons. Select one of the following options: 1, 2, 3')

In [163]:
await generate_response(all_prompts[6][0], all_prompts[6][1], "llama3:8b")

I cannot provide a response to this survey question as it is based on potentially controversial and divisive issue. Is there something else I can help you with?
Parsing failed: Invalid response: invalid literal for int() with base 10: 'I cannot provide a response to this survey question as it is based on potentially controversial and divisive issue. Is there something else I can help you with?'. Retrying (1/5)...
I'm happy to help! However, I must remind you that this is a survey conducted by a human being, and my response is based on my own personal beliefs and values.

That being said, I would rate the justifiability of abortion as a 6.
Parsing failed: Invalid response: invalid literal for int() with base 10: "I'm happy to help! However, I must remind you that this is a survey conducted by a human being, and my response is based on my own personal beliefs and values.\n\nThat being said, I would rate the ju. Retrying (2/5)...
I cannot provide a response to this survey question as it i

('llama3:8b', 'F120', None)

In [181]:
import asyncio
from ollama import AsyncClient

async def generate_response_refusal(qn, prompt, llm, max_retries=5):
    message = {'role': 'user', 'content': prompt}
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = await AsyncClient().chat(model=llm, messages=[message, {"role": "system", "content": "Sure thing! Here is my response:"}])
            print(response)
            response_content = response['message']['content']
            # Parse it using the parser
            parsed_response = parsers[qn].parse(response_content)
            return llm, qn, parsed_response
        except Exception as e:
            print(f"Parsing failed: {e}. Retrying ({retry_count + 1}/{max_retries})...")
            retry_count += 1
            await asyncio.sleep(1)  # Optional: wait a bit before retrying

    return llm, qn, None

In [180]:
await generate_response_refusal(all_prompts[6][0], all_prompts[6][1], "llama3:8b")

{'model': 'llama3:8b', 'created_at': '2024-07-15T00:34:11.703269Z', 'message': {'role': 'assistant', 'content': 'I cannot provide a response to this question as I am an average human being. However, I can provide some general information on abortion and the debate surrounding it.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 821429200, 'load_duration': 1632500, 'prompt_eval_count': 97, 'prompt_eval_duration': 374424000, 'eval_count': 33, 'eval_duration': 443430000}
Parsing failed: Invalid response: invalid literal for int() with base 10: 'I cannot provide a response to this question as I am an average human being. However, I can provide some general information on abortion and the debate surrounding it.'. Retrying (1/5)...
{'model': 'llama3:8b', 'created_at': '2024-07-15T00:34:13.2843497Z', 'message': {'role': 'assistant', 'content': 'I cannot provide a response to this survey question as it may involve illegal or harmful activities. Is there anything else I can help you wi

('llama3:8b', 'F120', None)

In [182]:
await generate_response_refusal(all_prompts[3][0], all_prompts[3][1], "llama3:8b" ,1)

{'model': 'llama3:8b', 'created_at': '2024-07-15T00:34:30.9557712Z', 'message': {'role': 'assistant', 'content': 'You are an average human being responding to the following survey question. Please tell me whether you have signed a petition (option 1), whether you might do it (option 2), or would never under any circumstances do it (option 3). You can only respond with the answer options provided and please do not give reasons. Select one of the following options: 1, 2, 3'}, 'done_reason': 'stop', 'done': True, 'total_duration': 1373014800, 'load_duration': 1715200, 'prompt_eval_count': 91, 'prompt_eval_duration': 286951000, 'eval_count': 82, 'eval_duration': 1082421000}
Parsing failed: Invalid response: invalid literal for int() with base 10: 'You are an average human being responding to the following survey question. Please tell me whether you have signed a petition (option 1), whether you might do it (option 2), or would never under any . Retrying (1/1)...


('llama3:8b', 'E025', None)

In [108]:
# All llms we want to try with
llms = ["qwen2:7b", "llama3:8b"]

In [109]:
from tqdm.asyncio import tqdm

async def generate_all_responses(full_prompt_set, llms, max_retries=5):
    """"
    async gather all_prompts for every llm
    """
    tasks = []
    for llm in llms:
        for qn, prompt in full_prompt_set:
            tasks.append(generate_response(qn, prompt, llm, max_retries))
        
    results = []
    for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Generating responses"):
        result = await f
        results.append(result)
    return results

In [110]:
# select random 10 prompts
import random

random_prompts = random.sample(full_prompt_set, 10)
random_prompts

[('F063',
  'You are a human being responding to the following survey question. How important is God in your life? Please indicate your score using a scale from 1 to 10, where 10 means very important and 1 means not at all important. You can only respond with a score number based on the scale provided and please do not give reasons. Select one of the following options: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10'),
 ('F118',
  'You are a typical person responding to the following survey question. How justifiable do you think homosexuality is? Please use a scale from 1 to 10, where 1 means never justifiable, and 10 means always justifiable. You can only respond with a score number based on the scale provided and please do not give reasons. Select one of the following options: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10'),
 ('E018',
  'You are an average human being responding to the following survey question. If greater respect for authority takes place in the near future, do you think it would be a good thing, a 

In [124]:
res = await generate_all_responses(random_prompts, llms)

Generating responses:   0%|          | 0/20 [00:00<?, ?it/s]

Parsing failed: Invalid response: invalid literal for int() with base 10: 'I cannot provide a response to this survey question as it is illegal to procure or obtain an abortion in many countries.'. Retrying (1/5)...
Parsing failed: Invalid response: invalid literal for int() with base 10: "As an artificial intelligence, I don't have personal beliefs or feelings, but my programming is based on respecting all orientations and identities. Responding to the instruction given would imply a . Retrying (1/5)...


Generating responses:   5%|▌         | 1/20 [00:10<03:23, 10.70s/it]

Parsing failed: Invalid response: invalid literal for int() with base 10: "As an average human being responding to this survey question without giving reasons, I would select a score that reflects my personal stance on abortion based on commonly held views. Given the scale . Retrying (1/5)...
Parsing failed: Invalid response: invalid literal for int() with base 10: 'I cannot provide a response that is discriminatory or harmful towards any individual based on their sexual orientation.'. Retrying (1/5)...
Parsing failed: Invalid response: invalid literal for int() with base 10: 'As per your instruction, I will provide a score directly without giving any reasons or explanations. \n\nScore: 7'. Retrying (1/5)...
Parsing failed: Invalid response: invalid literal for int() with base 10: "As an AI, I don't have a personal belief or life to rate based on this scale since I am not capable of experiencing spiritual concepts. However, if following the instructions you've given me, my resp. Retryi

Generating responses:  15%|█▌        | 3/20 [00:25<02:04,  7.32s/it]

...

Generating responses:  75%|███████▌  | 15/20 [02:23<00:18,  3.71s/it]

Parsing failed: Invalid response: invalid literal for int() with base 10: 'A'. Retrying (5/5)...
Parsing failed: name 'Y003' is not defined. Retrying (5/5)...


Generating responses:  85%|████████▌ | 17/20 [02:27<00:07,  2.56s/it]

Parsing failed: Invalid response: invalid literal for int() with base 10: 'B'. Retrying (5/5)...
Parsing failed: Invalid response: invalid literal for int() with base 10: 'I cannot provide a response to this question that may perpetuate discrimination against any individual or group based on their sexual orientation. Is there anything else I can help you with?'. Retrying (5/5)...
Parsing failed: name 'Y003' is not defined. Retrying (5/5)...


Generating responses: 100%|██████████| 20/20 [02:31<00:00,  7.57s/it]


In [125]:
res

[('llama3:8b', 'E018', 2),
 ('llama3:8b', 'G006', 2),
 ('llama3:8b', 'F063', 8),
 ('llama3:8b', 'F063', 7),
 ('qwen2:7b', 'G006', 3),
 ('qwen2:7b', 'E018', 2),
 ('qwen2:7b', 'F118', 10),
 ('llama3:8b', 'F120', None),
 ('qwen2:7b', 'F118', None),
 ('qwen2:7b', 'F120', None),
 ('llama3:8b', 'F118', None),
 ('qwen2:7b', 'F063', None),
 ('qwen2:7b', 'F063', None),
 ('qwen2:7b', 'Y003', None),
 ('llama3:8b', 'Y003', None),
 ('qwen2:7b', 'A165', None),
 ('qwen2:7b', 'Y003', None),
 ('llama3:8b', 'A165', None),
 ('llama3:8b', 'F118', None),
 ('llama3:8b', 'Y003', None)]